In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("train.csv")

In [3]:
df_test=pd.read_csv("test.csv")

In [4]:
df_result=pd.read_csv("gender_submission.csv")
df_result.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
dtype_df = df.dtypes.reset_index()
dtype_df

,index,0
0,PassengerId,int64
1,Survived,int64
2,Pclass,int64
3,Name,object
4,Sex,object
5,Age,float64
6,SibSp,int64
7,Parch,int64
8,Ticket,object
9,Fare,float64


In [9]:
import numpy as np

In [10]:
# Data sets cleaing
print('----Strat data cleaning ------------')
# Map Sex to a new column Gender as 'female': 0, 'male': 1
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)
# Delete Sex column from datasets
del df['Sex']
del df_test['Sex']

# Delete PassengerId column from train dataset only , needed in the test to map it while submiting.
del df['PassengerId']

# Delete Name column from datasets (No need for them in the analysis)
del df['Name']
del df_test['Name']
# Delete Ticket column from datasets  (No need for them in the analysis)
del df['Ticket']
del df_test['Ticket']

# Embarked has a lot of nan values, so i will remove it 
del df['Embarked']
del df_test['Embarked']

#manage Age
train_age_uq = df.Age.unique()
train_age_uq_has_nan = np.isnan(train_age_uq).any()

test_age_uq = df_test.Age.unique()
test_age_has_nan = np.isnan(test_age_uq).any()

if train_age_uq_has_nan == True or test_age_has_nan == True:
    train_random_ages = np.random.randint(df["Age"].mean() - df["Age"].std(),
                                          df["Age"].mean() + df["Age"].std(),
                                          size = df["Age"].isnull().sum())

    test_random_ages = np.random.randint(df_test["Age"].mean() - df_test["Age"].std(),
                                          df_test["Age"].mean() + df_test["Age"].std(),
                                          size = df_test["Age"].isnull().sum())

    df["Age"][np.isnan(df["Age"])] = train_random_ages
    df_test["Age"][np.isnan(df_test["Age"])] = test_random_ages
    df['Age'] = df['Age'].astype(int)
    df['Age']    = df['Age'].astype(int)

# Cabin has a lot of nan values, so i will remove it
del df['Cabin']
del df_test['Cabin']


# Fare
tuq = df.Fare.unique()
tuq_has_nan = np.isnan(tuq).any()

tsuq = df_test.Fare.unique()
tsuq_has_nan = np.isnan(tsuq).any()

if tuq_has_nan == True or tsuq_has_nan == True:
    df["Fare"].fillna(df["Fare"].median(), inplace=True)
    df_test["Fare"].fillna(df_test["Fare"].median(), inplace=True)

# convert to int
df['Fare']    = df['Fare'].astype(int)
df_test['Fare'] = df_test['Fare'].astype(int)

# Family

df['Family'] =  df["Parch"] + df["SibSp"]
df['Family'].loc[df['Family'] > 0] = 1
df['Family'].loc[df['Family'] == 0] = 0

df_test['Family'] =  df_test["Parch"] + df_test["SibSp"]
df_test['Family'].loc[df_test['Family'] > 0] = 1
df_test['Family'].loc[df_test['Family'] == 0] = 0

# drop Parch & SibSp
del df['SibSp']
del df_test['SibSp']
del df['Parch']
del df_test['Parch']

print(df.shape)
print(df_test.shape)

----Strat data cleaning ------------
(891, 6)
(418, 6)


C:\Users\Hamza Aziz\AppData\Local\conda\conda\envs\anderson\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hamza Aziz\AppData\Local\conda\conda\envs\anderson\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hamza Aziz\AppData\Local\conda\conda\envs\anderson\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value

In [11]:
x=df.drop(["Survived"], axis=1)
y=df.Survived
x_test=df_test.drop(["PassengerId"],1)
y_test=df_result.Survived

In [12]:
df.head()

,Survived,Pclass,Age,Fare,Gender,Family
0,0,3,22,7,1,1
1,1,1,38,71,0,1
2,1,3,26,7,0,0
3,1,1,35,53,0,1
4,0,3,35,8,1,0


In [13]:
df_test.head()

,PassengerId,Pclass,Age,Fare,Gender,Family
0,892,3,34.5,7,1,0
1,893,3,47.0,7,0,1
2,894,2,62.0,9,1,0
3,895,3,27.0,8,1,0
4,896,3,22.0,12,0,1


In [14]:
x = x.as_matrix()
y = y.as_matrix()
x_test = x_test.as_matrix()
y_test = y_test.as_matrix()


# Random Forests

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x, y)
Y_pred1 = random_forest.predict(x_test)
print(random_forest.score(x, y))

0.9607182940516273


# Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
logreg = LogisticRegression()
logreg.fit(x, y)
Y_pred = logreg.predict(x_test)
result = logreg.score(x, y)
print(result)

0.7901234567901234


# Support Vector Machines


In [19]:
from sklearn.svm import SVC, LinearSVC

In [20]:
svc = SVC()
svc.fit(x, y)
Y_pred = svc.predict(x_test)
print(svc.score(x, y))

0.8945005611672279


# Decision tree

In [21]:
from sklearn import tree
dt = tree.DecisionTreeClassifier()
dt = dt.fit(x, y)
Y_pred = dt.predict(x_test)
print(dt.score(x, y))

0.9607182940516273


# Naive Bayes

In [22]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb = gnb.fit(x, y)
Y_pred = gnb.predict(x_test)
print(gnb.score(x, y))

0.7744107744107744


# Multi-Layer Perceptron

In [23]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,100,100))
mlp = mlp.fit(x, y)
Y_pred = mlp.predict(x_test)
print(mlp.score(x, y))

0.7059483726150393


# Random forest gained the highest accuracy

In [24]:
print("Accuracy in %: ")
print(random_forest.score(x, y)*100)

Accuracy in %: 
96.07182940516273


In [25]:
submission = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": Y_pred1
    })
submission.to_csv('Submission.csv', index=False)
print('Exported')

Exported


In [26]:
pred=pd.read_csv("Submission.csv")

In [27]:
pred.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
PassengerId,418.0,1100.500000,120.810458,892.0,996.25,1100.5,1204.75,1309.0
Survived,418.0,0.366029,0.482295,0.0,0.00,0.0,1.00,1.0
